In [ ]:
# Depedensi 
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from normalisasi import *
from google_play_scraper import app, reviews #package library untuk scarape data review
import pandas as pd #mengolah data
from textblob import TextBlob #untuk Mengolahkata
from googletrans import Translator

# Download the NLTK corpora untuk menganalisis Sentiment dan support Textblob
import nltk
nltk.download('punkt')

Scarpping data Komen Tiktok beserta sentimentnya

In [ ]:
# Specify the app package name
app_package = 'com.twitter.android' # untuk mendapatkan package namenya ikutilah ling berikut ini
# https://www.techmesto.com/find-android-app-package-name/

# Get app details
app_info = app(app_package)

# Get reviews for the app
result, continuation_token = reviews(
    app_package,
    lang='en',
    country='us',
    count=100,
)

# Create a DataFrame to store the reviews
columns = ['Review', 'Sentiment']
data = []

# Populate the DataFrame with review data and sentiment analysis
for review in result:
    # Perform sentiment analysis using TextBlob
    analysis = TextBlob(review['content'])
    sentiment = 'Positive' if analysis.sentiment.polarity > 0 else 'Negative' if analysis.sentiment.polarity < 0 else 'Neutral'

    row_data = [
        review['content'],
        sentiment,
    ]
    data.append(row_data)

df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file
csv_filename = f'{app_info["title"]}_reviews_with_sentiment.csv'
df.to_csv(csv_filename, index=False)

print(f'DataFrame with sentiment analysis saved to {csv_filename}')

Translete to indonesia

In [ ]:
def translate_to_indonesian(text):
    translator = Translator()
    translation = translator.translate(text, dest='id')  # 'id' is the language code for Indonesian
    return translation.text

# Read the CSV file
file_path = 'TikTok_reviews_with_sentiment.csv'
df = pd.read_csv(file_path)

# Apply translation to the 'Text' column
df['Review'] = df['Review'].apply(translate_to_indonesian)

# Print the results
output_file_path = 'TikTok_reviews_with_sentiment_ind.csv'
df.to_csv(output_file_path, index=False)
print(f"\nResults exported to {output_file_path}")

Normalisasi data

In [ ]:
df = pd.read_csv('TikTok_reviews_with_sentiment_ind.csv') #read Data

texts = df['Review'].values
labels = df['Sentiment'].values

train_reviews = texts[:35000]
train_sentiments = labels[:35000]
test_reviews = texts[35000:]
test_sentiments = labels[35000:]


norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)
norm_train_reviews

In [ ]:
# Tokenize and preprocess the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(norm_train_reviews)
sequences = tokenizer.texts_to_sequences(norm_train_reviews)
X = pad_sequences(sequences, maxlen=100)

print("Vocabulary size={}".format(len(tokenizer.word_index)))
print("Number of Documents={}".format(tokenizer.document_count))
tokenizer.word_index


Sequence Normalization
Not all reviews are of same length. To handle this difference in length of reviews, we define a maximum length. For reviews which are smaller than this length, we pad them with zeros which longer ones are truncated

In [ ]:
MAX_SEQUENCE_LENGTH = 1000  
train_sequinces = tokenizer.texts_to_sequences(norm_train_reviews)
test_sequences = tokenizer.texts_to_sequences(norm_test_reviews)
# pad dataset to a maximum review length in words
X_train = pad_sequences(train_sequinces, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape

Encoding Labels
The dataset contains labels of the form positive/negative. The following step encodes the labels using sklearn's LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)



Prepare the Model
Since textual data is a sequence of words, we utilize 1D convolutions to scan through the sentences. The model first transforms each word into lower dimensional embedding/vector space followed by 1d convolutions and then passing the data through dense layers before the final layer for classification

In [ ]:
# Step 4: Build the CNN model
embedding_dim = 50  # Dimensionality of the word embeddings
filters = 128  # Number of filters in the convolutional layer
kernel_size = 5  # Size of the filters
hidden_dims = 100  # Number of neurons in the fully connected layer
max_words = 10000  # Max number of words in the vocabulary
max_len = 100  # Max number of words in a text

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model Training

In [ ]:
batch_size = 32
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Step 6: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


Model Evaluation

In [ ]:
# Step 6: Evaluate the model
y_pred_probabilities = model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Display and save the classification report

In [ ]:
# Display and save the classification report
unique_classes = label_encoder.classes_.astype(str)  # Ensure classes are in string format
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

# Save the classification report to a CSV file
class_report_dict = classification_report(y_test, y_pred, target_names=None, output_dict=True)
class_report_df = pd.DataFrame(class_report_dict).transpose()
class_report_df.to_csv('classification_report.csv')
print("Classification Report saved to 'classification_report.csv'")

In [ ]:
#Versi Dua

y_pred = model.predict(X_test)

# Step 8: Apply a threshold to convert probabilities to binary predictions
threshold = 0.5  # You can adjust this threshold based on your problem
y_pred_classes = (y_pred > threshold).astype(int).flatten()

# Step 9: Decode the labels back to their original form
decoded_pred_labels = label_encoder.inverse_transform(y_pred_classes)
decoded_true_labels = label_encoder.inverse_transform(y_test)

# Step 10: Create a DataFrame with original text, true labels, and predicted labels
results_data = {
    'Original_Text': tokenizer.sequences_to_texts(X_test),
    'True_Labels': decoded_true_labels,
    'Predicted_Labels': decoded_pred_labels
}

results_df = pd.DataFrame(results_data)

# Step 11: Save the results to a CSV file
results_df.to_csv('results.csv', index=False)

# Step 12: Print the classification report
print("Classification Report:")
print(classification_report(decoded_true_labels, decoded_pred_labels))